## L'ÎLE - Infocentre littéraire des écrivains québécois

Centre de documentation virtuel qui a pour mission de promouvoir la littérature québécoise. Il s’adresse aux étudiants, aux chercheurs, aux journalistes et à tous ceux qui s’intéressent aux écrivains québécois et à la littérature québécoise.

Regroupe des informations sur des écrivains québécois ou canadiens d’expression française, auteurs d’au moins deux livres d’au moins 48 pages, publiés par une maison d’édition reconnue, dont les livres appartiennent à l’un ou l’autre des genres suivants :
* le roman
* le récit
* la nouvelle
* le conte
* la poésie
* le théâtre
* l’essai ou l’étude
* la bande dessinée
* la biographie
* la littérature jeunesse


Le site fournit aussi une très grande liste à des sites externes (librairies, maisons d'édition, sites personnels des auteurs, associations, etc.)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import *
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [2]:
class Auteur:
    def __init__(self, nom = ""):
        self.nom = nom
        self.uri = ""
        self.bio = ""
        self.genres = []
        self.site = ""
        self.pseudonyme = ""

class Oeuvre:
    def __init__(self, id, titre = " "):
        self.id = id
        self.titre = titre
        self.auteurs = []
        self.lieuPublication = ""
        self.editeur = ""
        self.annee = ""
        self.isbn = ""
        

In [3]:
# Fonctions utiles

def extraireInfoOeuvre(id, contenu):
    oeuvre = Oeuvre(id)
#     print(contenu)

    
    # On extrait l'année et le titre
    annee = re.search(' \([0-9,-|]+\)',contenu[0])
    if annee:
        oeuvre.annee = annee.group(0)[2:-1]
        oeuvre.titre = contenu[0].replace(annee.group(0),'')

    else:
        oeuvre.titre = contenu[0]
        print('OUPS (pas d\'annéee dans le titre):',contenu[0])
               

    # On extrait le lieu de publication et l'éditeur
    # on cherche d'abord la chaîne qui commence par ", "
    edition = ""
    for c in contenu:
        if c.startswith(', '):
            edition = c
            break
    if edition: 
        edition = edition[2:]
        if ':' not in edition:
            print('OUPS (pas de lieu de publication):',edition)
            oeuvre.editeur = edition.strip()
        else:
            edition_infos = edition.split(':')
            oeuvre.lieuPublication = edition_infos[0].strip() 
            oeuvre.editeur = edition_infos[1].strip()
            
    else:
        print('OUPS (pas d\'édition spécifiée):', edition)

    for c in contenu:
        if c.strip().startswith('ISBN :'):
            oeuvre.isbn = c.strip()[7:]

    return oeuvre

def traiter_bio(texte):
    if texte.startswith('Notice biographique\n'):
        return texte[20:]
    else:
        return texte

def traiter_lieu_publication(texte):
    texte = texte.strip()
    if len(texte) > 2 and texte[0] == '[' and texte[-1] == ']':
        return texte[1:-1]
    else:
        return texte
    
   

# Extractions des hyperliens sur les auteurs de ILE

In [4]:
#launch url
url = "http://recif.litterature.org/recherche/"

# create a new Firefox session
driver = webdriver.Chrome('/home/ares/Downloads/chromedriver_linux64')
#driver.implicitly_wait(30)
driver.get(url)


WebDriverException: Message: 'chromedriver_linux64' executable may have wrong permissions. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
bouton = driver.find_element_by_class_name('btn_chercher')
bouton.click()

liens_auteurs = []
for a in driver.find_elements_by_css_selector('#colonne_resultats > dl > dd > ul > li > a '):
    liens_auteurs.append(a.get_attribute('href'))


In [ ]:
print(len(liens_auteurs))

# Extraction des infos sur les auteurs et leurs oeuvres

In [ ]:
auteurs = {}
oeuvres = {}



for a in liens_auteurs:
    auteur = Auteur()
    driver.get(a)

    # On extrait l'URI
    auteur.uri = a 

    # On extrait le nom de l'auteur
    auteur.nom = driver.find_element_by_css_selector('#ecrivain .desc > h1').text

    # On extrait la notice biographique
    try:
        bio = driver.find_element_by_css_selector('body div#wrapper div#contenu div#bio ') 
        suite = bio.find_element_by_css_selector('a.suite')
        if suite:
            try:
                suite.click()
            except ElementClickInterceptedException:
                pass
        auteur.bio = bio.text
    except NoSuchElementException:
        print('Cet auteur n\'a pas de bio:',auteur.nom)
        pass

    # On extrait les infos de la fiche
    fiche = driver.find_element_by_css_selector('div#contenu div#fiche') 
    items = fiche.find_elements_by_css_selector('div.desc :nth-child(2) > dt')

    position = 0
    print(auteur.nom)
    for i in items:
#         print(i.text)
        position += 1
        if i.text == "Genre(s) littéraire(s) :":
            genres = fiche.find_elements_by_css_selector('div.desc :nth-child(2) > dd:nth-of-type({0}) > a'.format(position))
            for g in genres:
#                 print(g.text)
                auteur.genres.append(g.text)
        if i.text == "Site Web :":
            site = fiche.find_element_by_css_selector('div.desc :nth-child(2) > dd:nth-of-type({0}) > a'.format(position))
#             print(site.text)  
            auteur.site = site.text
        if i.text == "Pseudo :":
            pseudo = fiche.find_element_by_css_selector('div.desc :nth-child(2) > dd:nth-of-type({0})'.format(position))
#             print(pseudo.text.strip())  
            auteur.pseudonyme = pseudo.text.strip()           
  

    liste_oeuvres = driver.find_elements_by_css_selector('div#oeuvres li')
    print(len(liste_oeuvres),'oeuvres')
    for o in liste_oeuvres:
        content = list(BeautifulSoup(o.get_property('innerHTML')).stripped_strings)
        oeuvre = extraireInfoOeuvre(o.get_attribute('id')[6:], content)

        # On ajoute le nom de l'auteur (dans la description de l'oeuvre, il se peut qu'il y ait 
        # d'autres auteurs, mais on les ignore pour l'instant)
        oeuvre.auteurs = [auteur.uri]
        oeuvres[oeuvre.id] = oeuvre
#         print(oeuvre.id)
#         print(oeuvre.titre)
        if len(oeuvre.annee) > 6:
            print(oeuvre.annee)

#         print(oeuvre.annee)
#         print(oeuvre.auteurs)
#         print(oeuvre.editeur)
#         print(oeuvre.isbn)
#         print('*****')
    print('---')

    auteurs[a] = auteur





In [ ]:
print(len(liens_auteurs))
print(len(auteurs), 'auteurs')
print(len(oeuvres), 'oeuvres')

# for a in auteurs.values():
#     print(a.uri)
# driver.get('http://recif.litterature.org/recherche/ecrivains/.-nathanael-825/')
# page_auteur = driver.find_element_by_id('bio')
# print(page_auteur.text)

In [ ]:

f = open('../Data/ILE/auteurs_ILE.csv',"w")
for a in auteurs.values():
    f.write("{}<>{}<>".format(a.uri, a.nom))
    f.write(",".join(a.genres) + "<>")
    f.write("{}<>{}<>{}".format(a.pseudonyme if a.pseudonyme else " " , a.site if a.site else " ", a.bio))
    f.write("\n-----\n")
f.close()

f = open('../Data/ILE/oeuvres_ILE.csv',"w")
for o in oeuvres.values():
    f.write("{}<>{}<>{}<>".format(o.id, o.titre.replace('\n',' '), o.annee))
    f.write(",".join(o.auteurs) + "<>")
    f.write("{}<>{}<>{}".format(o.editeur.replace('\n',' '), o.lieuPublication, o.isbn))
    f.write("\n")
f.close()